# Reddit API Data Pull

In [1]:
!pip install praw

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 191.0/191.0 kB 4.8 MB/s eta 0:00:00


In [13]:
import praw

# redacted; see personal auth details to use again 
user_agent = "",
reddit = praw.Reddit(client_id = "",
                    client_secret = "",
                     user_agent = user_agent)

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Subreddits scraped (case sensitive:



*   Psychology
*   Economics
*   Consulting
*   Jobs
*   remotework
*   humanresources
*   recruiting
*   tech   
*   business


To add -
*   Futurology
*   technology
*   Thedaily






In [4]:
# query terms for reddit comments

# Initialize empty lists to store data
all_post_data = []
all_comments_data = []

# Filter by these key words
query = ['artificial intelligence', 'job security', 'automation', 'robots',
         'ai adoption']

In [ ]:
import pandas as pd

# Initialize an empty list to store comment dataframes
all_comments_data = []

# list of reddit pages to query from
sub = ['IOPsychology']

# Filter by these key words
query = ['artificial intelligence', 'automation', 'robots',
         'ai adoption', 'AI','technological disruption',
         'technological advancement','technological changes', 'digitalization']
for s in sub:
    subreddit = reddit.subreddit(s)

    for item in query:
        comments_dict = {
            'comment_id': [],
            'comment_body': [],  # text in the comment
            'comment_link_id': [], # link to the comment
            'comment_utc': [], # time in utc
            'comment_subreddit': [],
            'comment_upvotes': []
        }

        # Search for submissions related to the query terms
        for submission in subreddit.search(item, sort='relevance', limit=100):
            submission.comments.replace_more(limit=0) # limit 0 so we only get comments to post, not threads

            # Collect comments associated with the submission
            for comment in submission.comments.list():
                comments_dict['comment_id'].append(comment.id)
                comments_dict['comment_body'].append(comment.body)
                comments_dict['comment_link_id'].append(comment.link_id)
                comments_dict['comment_utc'].append(comment.created_utc)
                comments_dict['comment_subreddit'].append(comment.subreddit)
                comments_dict['comment_upvotes'].append(comment.score)

        # Append the comments dataframe to the list
        all_comments_data.append(pd.DataFrame(comments_dict))

# Concatenate all comment dataframes into a single dataframe
all_comments_data_df = pd.concat(all_comments_data)


In [23]:
print(len(all_comments_data_df), 'before removing duplicates')

# remove duplicates
all_comments_data_df = all_comments_data_df.drop_duplicates('comment_id')

# new comment count after removing duplicates
print(len(all_comments_data_df), 'after removing duplicates')
all_comments_data_df.head()

1117 before removing duplicates
901 after removing duplicates


,comment_id,comment_body,comment_link_id,comment_utc,comment_subreddit,comment_upvotes
0,jaw9jzd,"Unless AI develops into true intelligence, no,...",t3_11i1l0h,1.677945e+09,IOPsychology,30.0
1,jawjpmb,So I think organizations are often bad at ass...,t3_11i1l0h,1.677949e+09,IOPsychology,9.0
2,jawz3vo,It will replace some jobs. An IO consulting co...,t3_11i1l0h,1.677955e+09,IOPsychology,5.0
3,jaw5muk,It depends. If AI continue to be things like C...,t3_11i1l0h,1.677943e+09,IOPsychology,8.0
4,jawrpip,Yes and no. ML can already read language for p...,t3_11i1l0h,1.677952e+09,IOPsychology,9.0


In [24]:
# Save to a single CSV file
all_comments_data_df.to_csv('/content/drive/MyDrive/Applied Paper /data/raw data from API /IOPsychology_subreddit_comments.csv', index=False)

In [25]:
len(all_comments_data_df)
all_comments_data_df.comment_body.head()

0    Unless AI develops into true intelligence, no,...
1    So I think organizations are often bad at  ass...
2    It will replace some jobs. An IO consulting co...
3    It depends. If AI continue to be things like C...
4    Yes and no. ML can already read language for p...
Name: comment_body, dtype: object

In [26]:
all_comments_data_df['date'] = pd.to_datetime(all_comments_data_df['comment_utc'], unit='s', utc=True)
all_comments_data_df['date'].min()
all_comments_data_df['date'].min()

Timestamp('2013-02-01 14:40:28+0000', tz='UTC')